In [1]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torch.optim import Adam
# from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader

import io
import imageio
from ipywidgets import widgets, HBox

# from models.dcgan import *
# from models.lstm import *
from data.MMNIST.mmnist import *
from utils.visualizations import *
from utils.TrainerBaseline import *
from models.vgg_baseline import *
from torchsummary import summary
# from utils.TrainerFP import *

%load_ext autoreload
%autoreload 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
import tensorboard
%load_ext tensorboard
%tensorboard --logdir=tboard_logs/kth_2_resnet_convlstm

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
models = VGGEncoder().to(device)
summary(models,(1,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]             640
       BatchNorm2d-2           [-1, 64, 64, 64]             128
         LeakyReLU-3           [-1, 64, 64, 64]               0
         ConvBlock-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]          36,928
       BatchNorm2d-6           [-1, 64, 64, 64]             128
         LeakyReLU-7           [-1, 64, 64, 64]               0
         ConvBlock-8           [-1, 64, 64, 64]               0
         MaxPool2d-9           [-1, 64, 32, 32]               0
           Conv2d-10          [-1, 128, 32, 32]          73,856
      BatchNorm2d-11          [-1, 128, 32, 32]             256
        LeakyReLU-12          [-1, 128, 32, 32]               0
        ConvBlock-13          [-1, 128, 32, 32]               0
           Conv2d-14          [-1, 128,

In [7]:
mmnist_data_dir = "data/MMNIST/" 
train_loader, val_loader,test_loader = MMNIST(mmnist_data_dir,batch_size = 1, seq_first = True, device=device)

ValueError: too many values to unpack (expected 3)

In [3]:
batch_size =1
sample_batch = next(iter(train_loader))
print("KTH Data Statistics:")
print(f"Train loader len: {len(train_loader)}, Total training sequences: {len(train_loader) * batch_size}")
print(f"Val loader len: {len(val_loader)}, Total validation sequences: {len(val_loader) * batch_size}")
print(f"Test loader len: {len(test_loader)}, Total testing sequences: {len(test_loader) * batch_size}")
print(f"Shape of batch: {sample_batch[0].shape}")

KTH Data Statistics:
Train loader len: 8000, Total training sequences: 8000
Val loader len: 1000, Total validation sequences: 1000
Test loader len: 2000, Total testing sequences: 2000
Shape of batch: torch.Size([1, 1, 64, 64])


In [4]:
trainer = TrainerBase(arch_type = "resnet", img_size = (1,64,64),device=device)

In [5]:
trainer.train(train_loader,val_loader,test_loader,num_epochs=11,device=device)

Epoch 2 Iter 10466: mse loss 31.90181 :  31%|███       | 2466/8000 [07:42<17:18,  5.33it/s]


KeyboardInterrupt: 

In [ ]:
from models.vgg_baseline import *
model = VGGEncoder()
model = model.to(device)
model

VGGEncoder(
  (vgg_block1): Sequential(
    (0): ConvBlock(
      (module): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
    (1): ConvBlock(
      (module): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convlstm1): predictor_lstm(
    (conv_lstms): ModuleList(
      (0): ConvLSTMCell(
        (conv): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      )
      (1): ConvLSTMCell(
        (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [ ]:
from torchsummary import summary
summary(model, (1,64,64))

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: [Sequential: 1-1, ConvBlock: 2-1, Sequential: 3-1, Conv2d: 4-1, BatchNorm2d: 4-2, LeakyReLU: 4-3, ConvBlock: 2-2, Sequential: 3-2, Conv2d: 4-4, BatchNorm2d: 4-5, LeakyReLU: 4-6, MaxPool2d: 2-3]

In [ ]:
torch.cuda.memory_stats(device=device)

OrderedDict()

In [ ]:
a, b = [5,[6,7,8]]
print(b)

In [ ]:
from models.vgg import VGGEncoder, VGGDecoder
model = VGGEncoder(in_size = (1, 64, 64))
model = model.to(device)
from torchsummary import summary
summary(model, (1,64,64))
# model

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─ConvBlock: 2-1                    [-1, 64, 64, 64]          --
|    |    └─Sequential: 3-1              [-1, 64, 64, 64]          768
|    └─ConvBlock: 2-2                    [-1, 64, 64, 64]          --
|    |    └─Sequential: 3-2              [-1, 64, 64, 64]          37,056
|    └─MaxPool2d: 2-3                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 128, 16, 16]         --
|    └─ConvBlock: 2-4                    [-1, 128, 32, 32]         --
|    |    └─Sequential: 3-3              [-1, 128, 32, 32]         74,112
|    └─ConvBlock: 2-5                    [-1, 128, 32, 32]         --
|    |    └─Sequential: 3-4              [-1, 128, 32, 32]         147,840
|    └─MaxPool2d: 2-6                    [-1, 128, 16, 16]         --
├─Sequential: 1-3                        [-1, 256, 8, 8]           --
|

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─ConvBlock: 2-1                    [-1, 64, 64, 64]          --
|    |    └─Sequential: 3-1              [-1, 64, 64, 64]          768
|    └─ConvBlock: 2-2                    [-1, 64, 64, 64]          --
|    |    └─Sequential: 3-2              [-1, 64, 64, 64]          37,056
|    └─MaxPool2d: 2-3                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 128, 16, 16]         --
|    └─ConvBlock: 2-4                    [-1, 128, 32, 32]         --
|    |    └─Sequential: 3-3              [-1, 128, 32, 32]         74,112
|    └─ConvBlock: 2-5                    [-1, 128, 32, 32]         --
|    |    └─Sequential: 3-4              [-1, 128, 32, 32]         147,840
|    └─MaxPool2d: 2-6                    [-1, 128, 16, 16]         --
├─Sequential: 1-3                        [-1, 256, 8, 8]           --
|

In [ ]:
from models.vgg import VGGEncoder, VGGDecoder
model = VGGDecoder(out_size = (1, 128, 128))
model = model.to(device)
# model

In [ ]:
from torchsummary import summary

summary(model, ((256,8,8),[(64,16,16)]))

Layer (type:depth-idx)                        Output Shape              Param #
├─ConvTranspose2d: 1-1                        [-1, 512, 8, 8]           590,336
├─Sequential: 1-2                             [-1, 256, 16, 16]         --
|    └─ConvBlock: 2-1                         [-1, 512, 8, 8]           --
|    |    └─Sequential: 3-1                   [-1, 512, 8, 8]           2,360,832
|    └─ConvBlock: 2-2                         [-1, 512, 8, 8]           --
|    |    └─Sequential: 3-2                   [-1, 512, 8, 8]           2,360,832
|    └─ConvTransposeBlock: 2-3                [-1, 256, 16, 16]         --
|    |    └─Sequential: 3-3                   [-1, 256, 16, 16]         2,097,920
├─Sequential: 1-3                             [-1, 128, 32, 32]         --
|    └─ConvBlock: 2-4                         [-1, 256, 16, 16]         --
|    |    └─Sequential: 3-4                   [-1, 256, 16, 16]         590,592
|    └─ConvBlock: 2-5                         [-1, 256, 16, 16] 

Layer (type:depth-idx)                        Output Shape              Param #
├─ConvTranspose2d: 1-1                        [-1, 512, 8, 8]           590,336
├─Sequential: 1-2                             [-1, 256, 16, 16]         --
|    └─ConvBlock: 2-1                         [-1, 512, 8, 8]           --
|    |    └─Sequential: 3-1                   [-1, 512, 8, 8]           2,360,832
|    └─ConvBlock: 2-2                         [-1, 512, 8, 8]           --
|    |    └─Sequential: 3-2                   [-1, 512, 8, 8]           2,360,832
|    └─ConvTransposeBlock: 2-3                [-1, 256, 16, 16]         --
|    |    └─Sequential: 3-3                   [-1, 256, 16, 16]         2,097,920
├─Sequential: 1-3                             [-1, 128, 32, 32]         --
|    └─ConvBlock: 2-4                         [-1, 256, 16, 16]         --
|    |    └─Sequential: 3-4                   [-1, 256, 16, 16]         590,592
|    └─ConvBlock: 2-5                         [-1, 256, 16, 16] 

In [ ]:
# model = DCGANEncoder(in_size = (1,128, 128))
# model = model.to(device)
import torchvision.models as models
from torchsummary import summary
model = models.vgg16()
model = model.to(device)

summary(model, (3,64, 64))
model

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 2, 2]           --
|    └─Conv2d: 2-1                       [-1, 64, 64, 64]          1,792
|    └─ReLU: 2-2                         [-1, 64, 64, 64]          --
|    └─Conv2d: 2-3                       [-1, 64, 64, 64]          36,928
|    └─ReLU: 2-4                         [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-5                    [-1, 64, 32, 32]          --
|    └─Conv2d: 2-6                       [-1, 128, 32, 32]         73,856
|    └─ReLU: 2-7                         [-1, 128, 32, 32]         --
|    └─Conv2d: 2-8                       [-1, 128, 32, 32]         147,584
|    └─ReLU: 2-9                         [-1, 128, 32, 32]         --
|    └─MaxPool2d: 2-10                   [-1, 128, 16, 16]         --
|    └─Conv2d: 2-11                      [-1, 256, 16, 16]         295,168
|    └─ReLU: 2-12                        [-1, 256, 16, 16]      

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
from models.resnet import *

model = Resnet18Encoder(in_size = (1,64,64))
model = model.to(device)
model


Resnet18Encoder(
  (c1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): ResidualBlockEncoder(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlockEncoder(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps

In [ ]:
summary(model, (1,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 64, 64, 64]          576
|    └─BatchNorm2d: 2-2                  [-1, 64, 64, 64]          128
|    └─LeakyReLU: 2-3                    [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-4                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 64, 32, 32]          --
|    └─ResidualBlockEncoder: 2-5         [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─LeakyReLU: 3-3               [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 32, 32]          128
|    |    └─LeakyReLU: 3-6               [-1, 64, 32, 32]          --
|  

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 64, 64, 64]          576
|    └─BatchNorm2d: 2-2                  [-1, 64, 64, 64]          128
|    └─LeakyReLU: 2-3                    [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-4                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 64, 32, 32]          --
|    └─ResidualBlockEncoder: 2-5         [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─LeakyReLU: 3-3               [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 32, 32]          128
|    |    └─LeakyReLU: 3-6               [-1, 64, 32, 32]          --
|  

In [ ]:
mmnist_data_dir = "data/MMNIST/" 
train_loader, val_loader,test_loader = MMNIST(mmnist_data_dir,batch_size = 20, seq_first = True, device=device)

In [ ]:
b = next(iter(val_loader))
b1 = next(iter(val_loader))
b.shape

In [ ]:
save_grid_batch(b, b1, nsamples = 5, text = "random", batch_first = False, show = True)

In [ ]:
save_pred_gifs(b, nsamples=5, text = "test", batch_first = False, show = True)

In [ ]:
from data.KTH.kth import *
from data.MMNIST.mmnist import *

kth_data_dir = "data/KTH/kth/"
mmnist_data_dir = "data/MMNIST/" 

batch_size = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader,test_loader = get_KTH(kth_data_dir, batch_size = 20, seq_first=True, frame_skip=20, device = device)

In [ ]:
batch_size = 20
sample_batch = next(iter(train_loader))
print("KTH Data Statistics:")
print(f"Train loader len: {len(train_loader)}, Total training sequences: {len(train_loader) * batch_size}")
print(f"Val loader len: {len(val_loader)}, Total validation sequences: {len(val_loader) * batch_size}")
print(f"Test loader len: {len(test_loader)}, Total testing sequences: {len(test_loader) * batch_size}")
print(f"Shape of batch: {sample_batch[0].shape}")

In [ ]:
for x in train_loader:
    print(x[0].shape)

In [ ]:
%load_ext autoreload
%autoreload 2
from utils.TrainerNP import *
trainer = TrainerNP(arch_type = "resnet", img_size = (1,128,128),device=device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
trainer.train(train_loader, val_loader,test_loader, device =device)

/home/user/ranaa0/video-prediction/code/data/KTH/kth.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch[i]['seq'] = torch.tensor(batch[i]['seq']) / 255.0
  0%|                                                                                                                                                                                                                                                                                                | 0/191 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 80.00 MiB (GPU 0; 10.76 GiB total capacity; 9.53 GiB already allocated; 66.50 MiB free; 9.55 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model = DCGANEncoder(in_size=(1,128,128))
model = model.to(device)
model

#**LSTM**

#**DCGAN encoder and decoder**

In [ ]:
x = torch.randn(100, 512, 4, 4)
y = torch.cat([x,x],1)

y.shape

In [ ]:
!pip3 install torch-summary

In [ ]:
from models.resnet import *
model = Resnet18Encoder(in_size = (1, 128, 128))
model = model.to(device)
model
from torchsummary import summary
summary(model, (1,128,128))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 64, 64]          --
|    └─Conv2d: 2-1                       [-1, 64, 128, 128]        576
|    └─BatchNorm2d: 2-2                  [-1, 64, 128, 128]        128
|    └─ReLU: 2-3                         [-1, 64, 128, 128]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 64, 64]          --
├─Sequential: 1-2                        [-1, 64, 64, 64]          --
|    └─ResidualBlockEncoder: 2-5         [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 64, 64]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-3                    [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 64, 64]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 64, 64]          128
|    └─ResidualBlockEncoder: 2-6         [-1, 64, 64, 64]          --
|  

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 64, 64]          --
|    └─Conv2d: 2-1                       [-1, 64, 128, 128]        576
|    └─BatchNorm2d: 2-2                  [-1, 64, 128, 128]        128
|    └─ReLU: 2-3                         [-1, 64, 128, 128]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 64, 64]          --
├─Sequential: 1-2                        [-1, 64, 64, 64]          --
|    └─ResidualBlockEncoder: 2-5         [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 64, 64]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-3                    [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 64, 64]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 64, 64]          128
|    └─ResidualBlockEncoder: 2-6         [-1, 64, 64, 64]          --
|  

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = SequentialClassifier(input_dim=28, emb_dim=64, hidden_dim=128, num_layers=2, mode="zeros")
model = latent_lstm(input_dim=64, out_dim=128, hidden_dim=128, num_layers=2, mode="zeros")
model = model.to(device)

In [ ]:
model

#**Moving MNIST Dataset**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
# from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader

import io
import imageio
from ipywidgets import widgets, HBox
%load_ext autoreload
%autoreload 2

In [ ]:
MovingMNIST = np.load('mnist_test_seq.npy')

In [ ]:
MovingMNIST.shape

In [ ]:
!pip3 install wget

In [ ]:
train_loader, val_loader,test_loader = MMNIST(seq_first = False,device=device)

In [ ]:
b = next(iter(val_loader))

In [ ]:
b.shape

In [ ]:
x =[]
for i in b:
    x.append(i)
x[0].shape

In [ ]:
for i in range(len(x)):
    x[i] = x[i].unsqueeze(0)
x[0].shape

In [ ]:
y = x[0]
for t in x[1:]:
    y = torch.cat([y,t],0)
y.shape

In [ ]:
from utils.visualizations import *
show_gif_batch(b)

In [ ]:
# Reverse process before displaying
future_batch = future_batch.cpu().numpy() * 255.0     
for video in future_batch.squeeze(1)[:5]:          # Loop over videos
    with io.BytesIO() as gif:
        imageio.mimsave(gif,video.astype(np.uint8),"GIF",fps=5)
        past = Box(children=[widgets.Image(value=gif.getvalue())], layout=box_layout)
        display(VBox([past]))

#**Training Fixed Prior Video Generation Model**

In [ ]:
#to be added in config
batch_size=100
past_frames = 10
future_frames = 10
z_dim = 10
g_dim = 128
lr = 0.002
beta = 0.0001
hidden_dim = 256

In [ ]:
encoder = DCGANEncoder()
decoder = DCGANDecoder()
encoder = encoder.to(device)
decoder = decoder.to(device)

predictor = predictor_lstm(g_dim+z_dim, g_dim, hidden_dim, num_layers=2)
posterior = latent_lstm(g_dim, z_dim, hidden_dim, num_layers=1)
predictor = predictor.to(device)
posterior = posterior.to(device)
# Decay LR by a factor of 0.1 every 5 epochs
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)
predictor_optimizer = torch.optim.Adam(predictor.parameters(), lr=lr, betas = (0.9, 0.999))
posterior_optimizer = torch.optim.Adam(posterior.parameters(), lr=lr, betas = (0.9, 0.999))
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr, betas = (0.9, 0.999))
decoder_optimizer = torch.optim.Adam(posterior.parameters(), lr=lr, betas = (0.9, 0.999))


In [ ]:
# Loss functions
mse_loss = nn.MSELoss()
# import torch.nn.functional as F
def kld_loss(mu, log_var):
    """
    Combined loss function for joint optimization of 
    prediction and ELBO
    """
    # mse = F.mse_loss(predicted, real, reduction="sum")
#     recons_loss = F.binary_cross_entropy(recons.view(b_size,-1), target.view(b_size,-1), reduction='sum')
    elbo = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    elbo /= batch_size
    # total_loss = mse + elbo*beta
    return elbo

In [ ]:
def train_epoch(x):
    predictor.zero_grad()
    posterior.zero_grad()
    encoder.zero_grad()
    decoder.zero_grad()

    # print("hello")

    h_seq = [encoder(x[i]) for i in range(past_frames+future_frames)]
    # print(h_seq[0][0].shape)
    mse =0
    kld = 0
    for i in range(1,past_frames+future_frames):
        h_target = h_seq[i][0]
        if i<past_frames:
            h, skip = h_seq[i-1]
        else:
            h = h_seq[i-1][0]
        # print("here")
        # print(h_target.shape)
        z_t, mu, log_var = posterior(h_target)
        # print("here")
        # print(z_t.shape)
        h_pred = predictor(torch.cat([h, z_t], 1))
        x_pred = decoder([h_pred, skip])
        mse+=mse_loss(x_pred,x[i])
        kld+=kld_loss(mu,log_var)

    total_loss = mse + kld*beta
    total_loss.backward()

    predictor_optimizer.step()
    posterior_optimizer.step()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return mse.data.cpu().numpy()/(past_frames+future_frames), kld.data.cpu().numpy()/(past_frames+future_frames)


In [ ]:
num_epochs = 300
for i in range(num_epochs):
    predictor.train()
    posterior.train()
    encoder.train()
    decoder.train() 
    epoch_mse =0
    epoch_kld=0
    epoch_loss =0

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for j, images in progress_bar:
        images = images.to(device)

        bs, nc, s, h, w = images.shape
        data = images.view(s, bs, nc, h, w)

        mse, kld = train_epoch(data)
        epoch_mse+=mse
        epoch_kld+=kld
        progress_bar.set_description(f"Epoch {i+1} : mse loss {epoch_mse:.5f}, kld los {epoch_kld: .5f} ")
